In [1]:
VALID_PTH = "zindi_data/ValDataset.csv"
PRDICTION_PTH = 'zindi_data/validation/prediction_cond-detr-50_THR0.000_IOU0.400_ID2146.csv'
THR_INF = 0.01
from zindi_code import CLS_MAPPER
CLS_MAPPER["NEG"] = 2
CLS_MAPPER

{'Trophozoite': 0, 'WBC': 1, 'NEG': 2}

In [2]:
import pandas as pd
import json
import os
from tqdm import tqdm

import numpy as np

In [3]:
valid_data = pd.read_csv(VALID_PTH)

predictions = pd.read_csv(PRDICTION_PTH)
predictions.sample(3)

,Image_ID,xmin,ymin,w,h,class,confidence,xmax,ymax
6306,id_jgst3boko3.jpg,559.66,2599.21,144.94,183.38,WBC,0.582,704.60,2782.59
9978,id_s6axfrvztf.jpg,2057.31,2636.99,81.66,105.09,Trophozoite,0.384,2138.97,2742.08
5630,id_5l5ws9o6mp.jpg,1294.54,1032.28,82.89,100.72,Trophozoite,0.400,1377.43,1133.00


In [4]:
valid_data.sample(3)

,Image_ID,class,confidence,ymin,xmin,ymax,xmax,width,height
549,id_j0zahs4ik5.jpg,Trophozoite,1.0,260,296,287,323,1920,1080
1516,id_147gpcegp6.jpg,Trophozoite,1.0,224,3340,344,3449,4160,3120
461,id_eibgf14l0w.jpg,Trophozoite,1.0,38,734,76,776,1920,1080


In [5]:
# valid_data = valid_data.rename(columns={
# 	"Image_ID": "image_id", "class": "category_id", "confidence": "score", "xmin": "x", "ymin": "y"
# })

# valid_data["w"] = valid_data["xmax"] - valid_data["x"]
# valid_data["h"] = valid_data["ymax"] - valid_data["y"]


# valid_data.sample(3)

In [6]:
predictions["Image_ID"].nunique(), valid_data["Image_ID"].nunique()

(275, 275)

In [7]:
len(set(predictions["Image_ID"].unique()).intersection(valid_data["Image_ID"].unique()))

275

In [8]:
predictions["Image_ID"].value_counts().head(10)

Image_ID
id_by6e6shi2z.jpg    70
id_qa70t0m4yc.jpg    66
id_taz7z5iuzk.jpg    66
id_q18tfhfneh.jpg    65
id_15rb0dgq66.jpg    64
id_qicgns95fp.jpg    64
id_ohdqwwlnpv.jpg    62
id_3xidcn77dw.jpg    61
id_zg9jc73sf7.jpg    61
id_0vaw9rsiro.jpg    60
Name: count, dtype: int64

In [9]:
valid_data["Image_ID"].value_counts().head(10)

Image_ID
id_15rb0dgq66.jpg    70
id_zg9jc73sf7.jpg    48
id_zdg96srigj.jpg    42
id_ndhdv3wuvg.jpg    39
id_i7nitsscaw.jpg    39
id_z0i61ad0tq.jpg    38
id_ilmjjeqpcy.jpg    37
id_by6e6shi2z.jpg    36
id_q18tfhfneh.jpg    33
id_087dra2apu.jpg    33
Name: count, dtype: int64

In [10]:
predictions["confidence"].describe()

count    10652.000000
mean         0.270664
std          0.187446
min          0.033000
25%          0.104000
50%          0.231500
75%          0.406000
max          0.842000
Name: confidence, dtype: float64

In [11]:
predictions.isna().sum()

Image_ID      0
xmin          0
ymin          0
w             0
h             0
class         0
confidence    0
xmax          0
ymax          0
dtype: int64

In [12]:
len(valid_data[valid_data["class"] == "NEG"])

70

In [13]:
predictions.columns

Index(['Image_ID', 'xmin', 'ymin', 'w', 'h', 'class', 'confidence', 'xmax',
       'ymax'],
      dtype='object')

In [14]:
valid_data.columns

Index(['Image_ID', 'class', 'confidence', 'ymin', 'xmin', 'ymax', 'xmax',
       'width', 'height'],
      dtype='object')

In [15]:
predictions.sample(5)

,Image_ID,xmin,ymin,w,h,class,confidence,xmax,ymax
9222,id_3j2sa06i6j.jpg,426.37,1474.98,154.22,206.01,WBC,0.638,580.59,1680.99
2277,id_6x0d145kam.jpg,2667.94,1748.94,82.97,109.88,Trophozoite,0.313,2750.91,1858.82
10164,id_yx8m83kug3.jpg,3570.59,3024.12,159.27,89.45,Trophozoite,0.039,3729.86,3113.57
1598,id_et97dmccl6.jpg,2203.18,1013.21,80.95,99.73,Trophozoite,0.438,2284.13,1112.94
2244,id_ctkvvkc84k.jpg,788.21,61.90,55.74,51.62,WBC,0.281,843.95,113.52


In [16]:
import torch

image_files: list[str] = valid_data["Image_ID"].unique().tolist()


def get_format_pred(file_id: str, data: pd.DataFrame, is_prediction, thr=THR_INF):
    pred = data[data["Image_ID"] == file_id].copy()
    pred = pred[pred["class"] != "NEG"]
    pred = pred[pred["confidence"] >= thr]
    if not len(pred):
        return dict(
        	boxes=torch.empty(size=[0, 4], dtype=torch.float).cuda(),
        	scores=torch.empty(size=[0], dtype=torch.float).cuda(),
        	labels=torch.empty(size=[0,], dtype=torch.int).cuda(),
        )
        return dict(
            boxes=torch.tensor([[0.0, 0.0, 0.0, 0.0]], dtype=torch.float).cuda(),
            scores=torch.tensor([1.0], dtype=torch.float).cuda(),
            labels=torch.tensor([CLS_MAPPER["NEG"]], dtype=torch.int).cuda(),
        )
    pred["w"] = pred["xmax"] - pred["xmin"]
    pred["h"] = pred["ymax"] - pred["ymin"]
    boxes = torch.tensor(
        pred[["xmin", "ymin", "w", "h"]].values, dtype=torch.float
    ).cuda()

    scores = torch.tensor(pred["confidence"].values, dtype=torch.float).cuda()
    labels = torch.tensor(
        [CLS_MAPPER[i] for i in pred["class"].values], dtype=torch.int
    ).cuda()
    result = dict(boxes=boxes, scores=scores, labels=labels)
    return result


print("Prediction Extraction")
preds = [get_format_pred(i, predictions, True) for i in image_files]
print("Expected Extraction")
expected = [get_format_pred(i, valid_data, False) for i in image_files]

Prediction Extraction
Expected Extraction


In [17]:
preds[0]

{'boxes': tensor([[ 709.3700,  997.5500,  163.1300,  198.6200],
         [1513.0900, 1375.3101,   91.3500,  114.8900],
         [1203.2800, 1152.2200,   84.4900,  105.6700],
         [ 758.3200,  203.1600,   88.6300,  113.2600],
         [ 350.4700,  488.7600,   87.4200,  108.5200],
         [1585.1000,  648.1800,   87.0400,  106.6400],
         [1841.8199,  914.0100,   90.5200,  112.2700],
         [ 464.8400, 1504.3800,   88.6100,  105.0900],
         [1368.6400, 2477.9500,   88.0400,  111.0600],
         [ 557.0200,  498.8100,   82.8400,  106.0100],
         [1487.6899, 1897.3500,   85.2100,  109.5900],
         [1305.2100,  428.2700,   84.4700,  115.9100],
         [2181.7800, 1031.8800,   84.4200,  110.9000],
         [ 208.4800, 2000.0699,   85.8400,  112.2800],
         [1055.4700,  518.5200,   85.0200,  106.0800],
         [ 975.9900, 1532.4000,   86.0100,  107.6800],
         [ 990.5800, 2105.8899,   84.6500,  107.4400],
         [2485.3601, 2068.4299,   84.7000,  107.9900],
 

In [18]:
expected[0]

{'boxes': tensor([[1744.,  925.,   79.,  116.],
         [2278.,  904.,  116.,  116.],
         [3060., 1141.,  100.,   95.],
         [1881., 1913.,   95.,  106.],
         [1702., 2489.,   85.,   90.],
         [2674., 1501.,  100.,   85.],
         [3023., 1675.,  106.,  111.],
         [2848., 1353.,   90.,  106.],
         [ 470., 1765.,  116.,  122.],
         [2943., 1226.,   95.,  106.],
         [2933.,  671.,   95.,   85.],
         [1311.,  534.,   90.,  100.],
         [ 254., 2003.,  100.,  111.],
         [ 941.,  206.,   90.,  106.],
         [2949., 1041.,   90.,  106.],
         [1649., 2669.,   90.,   95.],
         [ 460.,  523.,   90.,   85.],
         [ 925.,  988.,  169.,  206.],
         [1887., 1395.,   95.,   90.],
         [3012.,  772.,  106.,  100.],
         [2743., 1046.,   74.,   85.]], device='cuda:0'),
 'scores': tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1.], device='cuda:0'),
 'labels': tensor([0, 

In [19]:
from torchmetrics.detection import MeanAveragePrecision

metric = MeanAveragePrecision(
    iou_type="bbox",
    box_format="xywh",
    class_metrics=True,
    extended_summary=False,
    # backend="faster_coco_eval",
)


metric.update(preds=preds, target=expected)

from pprint import pprint

pprint(metric.compute())

{'classes': tensor([0, 1], dtype=torch.int32),
 'map': tensor(0.0045),
 'map_50': tensor(0.0194),
 'map_75': tensor(0.0007),
 'map_large': tensor(6.7998e-05),
 'map_medium': tensor(0.0087),
 'map_per_class': tensor([0.0004, 0.0086]),
 'map_small': tensor(0.0116),
 'mar_1': tensor(0.0043),
 'mar_10': tensor(0.0323),
 'mar_100': tensor(0.0423),
 'mar_100_per_class': tensor([0.0143, 0.0703]),
 'mar_large': tensor(0.0087),
 'mar_medium': tensor(0.0558),
 'mar_small': tensor(0.0162)}


In [20]:
expected[128]

{'boxes': tensor([[1277.,  127.,   85.,   70.],
         [1583.,  878.,   62.,   61.],
         [ 755.,  815.,   59.,   64.],
         [ 929.,  165.,   38.,   34.],
         [ 864.,  375.,   96.,   79.],
         [1430.,   66.,   26.,   25.],
         [ 473.,  513.,   32.,   38.],
         [1110.,  131.,   28.,   28.],
         [ 908.,  291.,   64.,   59.],
         [1394.,  600.,   28.,   26.],
         [1226.,  269.,   32.,   34.],
         [1432.,  110.,   30.,   32.],
         [ 653.,  815.,   30.,   32.],
         [1131.,  161.,   62.,   81.]], device='cuda:0'),
 'scores': tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        device='cuda:0'),
 'labels': tensor([1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1], device='cuda:0',
        dtype=torch.int32)}

In [21]:
preds[128]

{'boxes': tensor([[ 748.3300,  761.7400,   67.8500,   69.5700],
         [1572.8800,  822.8400,   65.1300,   58.0800],
         [ 901.1900,  269.3500,   69.1800,   64.0900],
         [1269.7800,  116.5600,   96.8900,   72.6100],
         [ 991.0400,  450.8000,   39.3400,   35.1800],
         [1100.7900,  118.2600,   38.0400,   36.3000],
         [1044.0500,  627.8600,   37.5500,   34.1200],
         [1128.5300,  154.0300,   72.2700,   77.2700],
         [ 762.2400,  286.0300,   37.2100,   33.6800],
         [ 855.5900,  943.3900,   36.3200,   34.9900],
         [1228.4800,  249.4800,   36.8000,   35.6400],
         [ 677.1600,  598.7000,   66.2100,   52.3700],
         [1389.9800,  559.6100,   36.6100,   33.5300],
         [ 647.4300,  767.2100,   36.5000,   32.8900],
         [ 702.5400,  291.8900,   37.7400,   33.7600],
         [ 982.9900,  531.8200,   40.1700,   38.7800],
         [1209.8900,  645.9700,   36.8900,   34.3900],
         [ 469.5600,  483.2600,   36.5200,   32.7400],
 

In [22]:
from tqdm import tqdm

thrs = np.linspace(0.0, 0.95, num=15)
best_score = 0
best_thr = 0

for thr in tqdm(thrs):
    metric = MeanAveragePrecision(
        iou_type="bbox",
        box_format="xywh",
        class_metrics=True,
        extended_summary=False,
        # backend="faster_coco_eval",
    )
    preds = [get_format_pred(i, predictions, True, thr) for i in image_files]
    expected = [get_format_pred(i, valid_data, False, thr) for i in image_files]
    metric.update(preds=preds, target=expected)
    scores = metric.compute()
    if scores["map_50"] > best_score:
        best_score = scores["map_50"]
        best_thr = thr
        print(best_thr, best_score)

  0%|          | 0/15 [00:00<?, ?it/s]

  7%|▋         | 1/15 [00:05<01:13,  5.22s/it]

0.0 tensor(0.0194)


100%|██████████| 15/15 [00:44<00:00,  2.96s/it]


In [23]:
print(best_thr, best_score)

0.0 tensor(0.0194)


In [24]:
thrs

array([0.        , 0.06785714, 0.13571429, 0.20357143, 0.27142857,
       0.33928571, 0.40714286, 0.475     , 0.54285714, 0.61071429,
       0.67857143, 0.74642857, 0.81428571, 0.88214286, 0.95      ])

In [25]:
thr = 0.05

metric = MeanAveragePrecision(
    iou_type="bbox",
    box_format="xywh",
    class_metrics=True,
    extended_summary=False,
    backend="faster_coco_eval",
)
preds = [get_format_pred(i, predictions, True, thr) for i in image_files]
expected = [get_format_pred(i, valid_data, False, thr) for i in image_files]
metric.update(preds=preds, target=expected)
scores = metric.compute()

In [26]:
scores

{'map': tensor(0.0045),
 'map_50': tensor(0.0194),
 'map_75': tensor(0.0007),
 'map_small': tensor(0.0116),
 'map_medium': tensor(0.0087),
 'map_large': tensor(6.7998e-05),
 'mar_1': tensor(0.0043),
 'mar_10': tensor(0.0323),
 'mar_100': tensor(0.0423),
 'mar_small': tensor(0.0162),
 'mar_medium': tensor(0.0558),
 'mar_large': tensor(0.0087),
 'map_per_class': tensor([0.0004, 0.0086]),
 'mar_100_per_class': tensor([0.0143, 0.0703]),
 'classes': tensor([0, 1], dtype=torch.int32)}